In [ ]:

import sys
import os

py_file_location = "Dirección archivos.py AdapNet++"
sys.path.append(os.path.abspath(py_file_location))

import argparse
import datetime
import importlib
import re
import numpy as np
import tensorflow as tf
import yaml
from dataset.helper import *

In [ ]:
num_clases='Número de clases'

batch_size='Tamño de lote

In [ ]:
def get_test_data():
    iterator = get_test_batch()
    dataA, label, dataB = iterator.get_next()
    return [dataA, label, dataB], iterator

def get_test_batch():
    filenames = 'Dirección tfrecord conjunto evaluación'
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.map(lambda x: parser(x, num_clases))
    dataset = dataset.batch(batch_size)
    iterator = dataset.make_initializable_iterator()
    return iterator

def compute_output_matrix(label_max, pred_max, output_matrix):
    for i in range(output_matrix.shape[0]):
        temp = pred_max == i
        temp_l = label_max == i
        tp = np.logical_and(temp, temp_l)
        temp[temp_l] = True
        fp = np.logical_xor(temp, temp_l)
        temp = pred_max == i
        temp[fp] = False
        fn = np.logical_xor(temp, temp_l)
        output_matrix[i, 0] += np.sum(tp)
        output_matrix[i, 1] += np.sum(fp)
        output_matrix[i, 2] += np.sum(fn)

    return output_matrix

In [ ]:
def cargar_modelo(ckpt_path,sess):
  sess.run(tf.global_variables_initializer())
  import_variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
  print('total_variables_loaded:', len(import_variables))
  saver = tf.train.Saver(import_variables)
  saver.restore(sess, ckpt_path)
  sess.run(iterator.initializer)



def calculo_mIoU(sess):
  output_matrix = np.zeros([13, 3])
  step = 0
  while 1:
      try:
          img, label, img1 = sess.run([data_list[0], data_list[1], data_list[2]])
          feed_dict = {images_pl : img, images1_pl : img1}
          probabilities = sess.run([model.softmax], feed_dict=feed_dict)
          prediction = np.argmax(probabilities[0], 3)
          gt = np.argmax(label, 3)
          #prediction[gt == 0] = 0  Activar cuando el fondo ocupe solo una pequeña parte de la imagen.
          output_matrix = compute_output_matrix(gt, prediction, output_matrix)
          step += 1

      except tf.errors.OutOfRangeError:
          break
  return compute_iou(output_matrix)

In [ ]:
module = importlib.import_module('models.' + 'SSMA')
model_func = getattr(module, 'SSMA')
data_list, iterator = get_test_data()
model = model_func(num_classes=num_clases, training=False)
images_pl = tf.placeholder(tf.float32, [None, 384, 768, 3])
images1_pl = tf.placeholder(tf.float32, [None, 384, 768, 3])
model.build_graph(images_pl, images1_pl)
config1 = tf.ConfigProto()
config1.gpu_options.allow_growth = True

MIOUs=[]
path='Dirección carpeta con todos los checkpoints'
#Los ckpts empiezan en 9
index=9
while 1:
  try:
    path_ckpt='Dirección ckpt sin introducir el número (Ej. C://User...//model_RGB_city_uma.ckpt-)'
    num=str(index)
    path_ckpt="".join((path_ckpt,num))
    sess = tf.Session(config=config1)
    cargar_modelo(path_ckpt,sess)
    MIOUs.append(calculo_mIoU(sess))
    #Si se guarda un ckpt cada 10 iteracines
    index=index+10
  except tf.errors.OutOfRangeError:
      print ('No hay mas ckpts')
      break

In [ ]:
import matplotlib.pyplot as plt

ejeX=list(range(0,len(MIOUs)))

plt.plot(ejeX,MIOUs)